<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EB%89%B4%EC%8A%A4_%EC%A0%95%EB%B3%B4%EC%99%80_%ED%86%B5%EA%B3%84_%EC%A0%95%EB%B3%B4%2C_%EB%8C%93%EA%B8%80_%EC%A0%95%EB%B3%B4_%EC%B7%A8%ED%95%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 실습3
# 각 뉴스의 댓글 수를 스크랩해서 저장해주세요.


## requests로 정적데이터 인지 한번 확인해보기
url = 'https://n.news.naver.com/article/015/0004891823?ntype=RANKING'


import requests


## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 네이버 랭킹 뉴스 웹페이지 요청. 성공하면 응답이옴(r)
r = requests.get(url, headers=headers)

## 뷰티풀숩으로 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

target = soup.select_one('.u_cbox_count')

## 결과가 None이 나옴. 동적 데이터임을 알 수 있음.


In [ ]:
# 셀레니움으로 가져오기

# 패키지 불러오기
import requests
import selenium
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless')
service = ChromeService(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

## 댓글 수 하나 가져오기 테스트
driver.get('https://n.news.naver.com/article/015/0004891823?ntype=RANKING')

count = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".u_cbox_count"))
    )

## 성공

In [ ]:
## 기존 모듈을 활용하여 뉴스들의 댓글 정보 가져와서 저장하기

import news_util as ut

def get_news_by_journal(journal) :
    target_link = journal['link'] + '&date=20230907'

    page = ut.get_page_by_url(target_link)
    ranking_news = ut.get_rankingnews_by_page(page)

    return ranking_news

sbs = ut.get_journal_by_name('SBS')
kbs = ut.get_journal_by_name('KBS')
mbc = ut.get_journal_by_name('MBC')

journal_list = [sbs, kbs, mbc]

for journal in journal_list :
    ranking_news = get_news_by_journal(journal)

    repl_cnt = {}
    repl_cnt_list = []
    for news in ranking_news:
        try :
            driver.get(news['link'])
            count = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".u_cbox_count"))
                )

            repl_cnt['count'] = count.text
            repl_cnt['id'] = news['id']

            repl_cnt_list.append(repl_cnt)
        except :
            print("문제 발생.")

    print(f"{journal['name']} 작업 완료")

    file_path = journal['name'] + '_replCnt.json'
    ut.save_to_json(file_path, repl_cnt_list)

In [ ]:
## 스크랩한 뉴스 데이터를 판다스로 불러와서 취합하기
import pandas as pd

kbs_news = pd.read_json('data/KBS_news.json')
sbs_news = pd.read_json('data/SBS_news.json')
mbc_news = pd.read_json('data/MBC_news.json')

news_df = pd.concat([kbs_news, sbs_news, mbc_news])
news_df

In [ ]:
## 스크랩한 통계 데이터를 판다스로 불러와서 취합하기

kbs_stat = pd.read_json('data/KBS_stat.json')
sbs_stat = pd.read_json('data/SBS_stat.json')
mbc_stat = pd.read_json('data/MBC_stat.json')

stat_df = pd.concat([kbs_stat, sbs_stat, mbc_stat])

stat_df

In [ ]:
## 간이 문제. 뉴스 정보와 통계 정보를 id를 기준으로 merge해주세요.